## Connecting To Spark

In [1]:
import plotly as pl

In [2]:
import matplotlib.pyplot as plt # a module for plotting histograms
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.types import *

# define spark session
sc = pyspark.SparkContext("local[*]", "temp")
print("Made it here")


Made it here


## Reading in Moby Dick

In [3]:
spark = SparkSession(sc)

file_book = open("/spring2021/project1/comparison/Herman Melville - Moby.txt",'r')

lines = file_book.readlines()[845:]

dataColumns = ["line"]

mobyDf = pd.DataFrame(lines, columns = ['Line'])
mobyDf2 = spark.createDataFrame(mobyDf)


mobyDf2.createOrReplaceTempView("moby")



new_df2 = spark.sql('SELECT Line, LENGTH(Line)AS LOS, (LENGTH(Line) -  LENGTH(REPLACE(Line,"a",""))) AS a, (LENGTH(Line) -  LENGTH(REPLACE(Line,"e",""))) AS e, (LENGTH(Line) -  LENGTH(REPLACE(Line,"i",""))) AS i, (LENGTH(Line) -  LENGTH(REPLACE(Line,"o",""))) AS o, (LENGTH(Line) -  LENGTH(REPLACE(Line,"u",""))) AS u,(LENGTH(Line) -  LENGTH(REPLACE(Line,"A",""))) AS Ac , (LENGTH(Line) -  LENGTH(REPLACE(Line,"E",""))) AS Ec,(LENGTH(Line) -  LENGTH(REPLACE(Line,"I",""))) AS Ic, (LENGTH(Line) -  LENGTH(REPLACE(Line,"O",""))) AS Oc,(LENGTH(Line) -  LENGTH(REPLACE(Line,"U",""))) AS Uc FROM moby')
new_df2.show(10)



+--------------------+---+---+---+---+---+---+---+---+---+---+---+
|                Line|LOS|  a|  e|  i|  o|  u| Ac| Ec| Ic| Oc| Uc|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+
|                   
|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|Call me Ishmael. ...| 69|  5|  8|  3|  4|  0|  0|  0|  1|  0|  0|
|little or no mone...| 70|  3|  6|  5|  5|  2|  0|  0|  0|  0|  0|
|on shore, I thoug...| 72|  6|  6|  2|  5|  3|  0|  0|  2|  0|  0|
|of the world. It ...| 63|  4|  5|  3|  4|  0|  0|  0|  2|  0|  0|
|regulating the ci...| 70|  3|  6|  6|  3|  3|  0|  0|  1|  0|  0|
|the mouth; whenev...| 72|  2|  9|  4|  3|  2|  0|  0|  0|  0|  0|
|I find myself inv...| 66|  4|  5|  5|  4|  3|  0|  0|  1|  0|  0|
|bringing up the r...| 70|  4| 12|  3|  1|  2|  0|  0|  1|  0|  0|
|my hypos get such...| 71|  5|  5|  2|  4|  3|  0|  0|  0|  0|  0|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+
only showing top 10 rows



## Data Science Libraries

In [4]:
import plotly.express as px

## Data Munging

In [33]:
new_df2.createOrReplaceTempView("vowels")

# aggregate columns
new_df3 = spark.sql('SELECT SUM(a) AS Total_A_lwr, SUM(e) AS Total_E_lwr, SUM(i) AS Total_I_lwr, SUM(o) AS Total_O_lwr, SUM(u) AS Total_U_lwr, SUM(Ac) AS Total_A_upr, SUM(Ec) AS Total_E_upr, SUM(Ic) AS Total_I_Ic, SUM(Oc) AS Total_O_upr, SUM(Uc) AS Total_U_upr  FROM vowels')
new_df3.show()

+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+
|Total_A_lwr|Total_E_lwr|Total_I_lwr|Total_O_lwr|Total_U_lwr|Total_A_upr|Total_E_upr|Total_I_Ic|Total_O_upr|Total_U_upr|
+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+
|      75268|     116129|      62270|      68867|      26601|       2175|        629|      3181|        574|        143|
+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+



In [34]:
mobyDickFinal = new_df3.toPandas()

In [35]:
mobyDickFinal

,Total_A_lwr,Total_E_lwr,Total_I_lwr,Total_O_lwr,Total_U_lwr,Total_A_upr,Total_E_upr,Total_I_Ic,Total_O_upr,Total_U_upr
0,75268,116129,62270,68867,26601,2175,629,3181,574,143


In [91]:
lwr = mobyDickFinal.iloc[:, 0:5]
lwr = lwr.rename(columns = {"Total_A_lwr": "a", "Total_E_lwr" : "e", "Total_I_lwr" : "i", "Total_O_lwr" : "o", "Total_U_lwr" : "u" })
lwr = lwr.T
lwr = lwr.rename(columns = { 0:"Count"})
vows = ['a','e','i','o','u']

lwr['Vowels'] = vows
lwr.reset_index(drop = True, inplace = True)
n = lwr.sum()[0]
lwr = lwr.assign(prop = lambda dataframe: dataframe['Count'].map(lambda Count: (Count / n) * 100 ))
lwr

,Count,Vowels,prop
0,75268,a,21.558423
1,116129,e,33.261919
2,62270,i,17.835508
3,68867,o,19.725035
4,26601,u,7.619116


In [108]:
upr = mobyDickFinal.iloc[:, 5:10]
upr = upr.rename(columns = {"Total_A_upr" : "A", "Total_E_upr" : "E", "Total_I_Ic" : "I", "Total_O_upr" : "O", "Total_U_upr" : "U"})
upr = upr.T
upr = upr.rename(columns = {0: "Count"})
vows2 = ['A', 'E', 'I', 'O','U']
upr['Vowels'] = vows2
upr.reset_index(drop = True, inplace = True)
n2 = upr.sum()[0]
upr = upr.assign(prop = lambda dataframe: dataframe['Count'].map(lambda Count: (Count / n2) * 100 ))

upr

,Count,Vowels,prop
0,2175,A,32.452999
1,629,E,9.385258
2,3181,I,47.463444
3,574,O,8.564608
4,143,U,2.133691


## Creating Plots

In [96]:
fig = px.bar(lwr, x = 'Vowels', y = 'Count', text =lwr['prop'].apply(lambda x: '{0:1.2f}%'.format(x)))
fig.update_layout(yaxis_tickformat = ',', title = {'text': 'Distribution of Lower-case Vowels in Moby Dick', 'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top' })
fig.show()

In [120]:
fig = px.bar(upr, x = 'Vowels', color = 'Vowels', color_discrete_sequence = ['#00CC96'], y = 'Count', text =upr['prop'].apply(lambda x: '{0:1.2f}%'.format(x)))
fig.update_layout(showlegend = False, yaxis_tickformat = ',', title = {'text': 'Distribution of Upper-case Vowels in Moby Dick', 'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top' })
fig.show()

In [102]:
upr = upr.rename(columns = {"Vowels":"Vowels_U", "Count" : "Count_U", "prop":"prop_U"})
upr

,Count_U,Vowels_U,prop_U
0,2175,A,32.452999
1,629,E,9.385258
2,3181,I,47.463444
3,574,O,8.564608
4,143,U,2.133691


In [103]:
res = pd.concat([lwr,upr], axis = 1, join = "inner")
res

,Count,Vowels,prop,Count_U,Vowels_U,prop_U
0,75268,a,21.558423,2175,A,32.452999
1,116129,e,33.261919,629,E,9.385258
2,62270,i,17.835508,3181,I,47.463444
3,68867,o,19.725035,574,O,8.564608
4,26601,u,7.619116,143,U,2.133691


In [106]:
import plotly.graph_objects as go

fig = go.Figure(data = [go.Bar(name = "Lower", x = res['Vowels'], y = res['Count']), go.Bar(name = "Upper", x = res['Vowels_U'], y = res['Count_U'])])

fig.update_layout(barmode = 'group')
fig.show()

In [121]:
#'Conan Doyle - Sherlock.txt'

ModuleNotFoundError: No module named 'dash'

## Sherlock Holmes

In [122]:
file_book = open("/spring2021/project1/comparison/Conan Doyle - Sherlock.txt",'r')
# start at 63
lines = file_book.readlines()[63:]
dataColumns = ["line"]


sherlockDf = pd.DataFrame(lines, columns = ['Line'])
sherlockDf2 = spark.createDataFrame(sherlockDf)

sherlockDf2.createOrReplaceTempView("sherlock")

new_df2 = spark.sql('SELECT Line, LENGTH(Line)AS LOS, (LENGTH(Line) -  LENGTH(REPLACE(Line,"a",""))) AS a, (LENGTH(Line) -  LENGTH(REPLACE(Line,"e",""))) AS e, (LENGTH(Line) -  LENGTH(REPLACE(Line,"i",""))) AS i, (LENGTH(Line) -  LENGTH(REPLACE(Line,"o",""))) AS o, (LENGTH(Line) -  LENGTH(REPLACE(Line,"u",""))) AS u,(LENGTH(Line) -  LENGTH(REPLACE(Line,"A",""))) AS Ac , (LENGTH(Line) -  LENGTH(REPLACE(Line,"E",""))) AS Ec,(LENGTH(Line) -  LENGTH(REPLACE(Line,"I",""))) AS Ic, (LENGTH(Line) -  LENGTH(REPLACE(Line,"O",""))) AS Oc,(LENGTH(Line) -  LENGTH(REPLACE(Line,"U",""))) AS Uc FROM sherlock')
new_df2.show(10)


+--------------------+---+---+---+---+---+---+---+---+---+---+---+
|                Line|LOS|  a|  e|  i|  o|  u| Ac| Ec| Ic| Oc| Uc|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+
|To Sherlock Holme...| 70|  5|  7|  2|  5|  0|  0|  0|  1|  0|  0|
|mention her under...| 63|  3| 10|  3|  2|  1|  0|  0|  1|  0|  0|
|predominates the ...| 71|  4|  9|  2|  6|  0|  0|  0|  1|  0|  0|
|akin to love for ...| 71|  5|  6|  3|  6|  1|  2|  0|  1|  0|  0|
|were abhorrent to...| 68|  5|  7|  4|  3|  1|  0|  0|  0|  0|  0|
|was, I take it, t...| 70|  6|  7|  4|  3|  0|  0|  0|  1|  0|  0|
|the world has see...| 69|  6|  8|  2|  3|  2|  0|  0|  0|  0|  0|
|false position. H...| 72|  4|  9|  5|  6|  0|  0|  0|  0|  0|  0|
|and a sneer. They...| 71|  4| 12|  2|  3|  0|  0|  0|  0|  0|  0|
|drawing the veil ...| 69|  4|  6|  5|  4|  1|  0|  0|  0|  0|  0|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+
only showing top 10 rows



In [123]:
new_df2.createOrReplaceTempView("vowels")

# aggregate columns
new_df3 = spark.sql('SELECT SUM(a) AS Total_A_lwr, SUM(e) AS Total_E_lwr, SUM(i) AS Total_I_lwr, SUM(o) AS Total_O_lwr, SUM(u) AS Total_U_lwr, SUM(Ac) AS Total_A_upr, SUM(Ec) AS Total_E_upr, SUM(Ic) AS Total_I_Ic, SUM(Oc) AS Total_O_upr, SUM(Uc) AS Total_U_upr  FROM vowels')
new_df3.show()

+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+
|Total_A_lwr|Total_E_lwr|Total_I_lwr|Total_O_lwr|Total_U_lwr|Total_A_upr|Total_E_upr|Total_I_Ic|Total_O_upr|Total_U_upr|
+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+
|      35276|      54499|      27349|      34432|      13483|        813|        332|      3843|        362|         88|
+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+



In [124]:
sherlockFinal = new_df3.toPandas()
sherlockFinal

,Total_A_lwr,Total_E_lwr,Total_I_lwr,Total_O_lwr,Total_U_lwr,Total_A_upr,Total_E_upr,Total_I_Ic,Total_O_upr,Total_U_upr
0,35276,54499,27349,34432,13483,813,332,3843,362,88


In [126]:
lwr = sherlockFinal.iloc[:, 0:5]
lwr = lwr.rename(columns = {"Total_A_lwr": "a", "Total_E_lwr" : "e", "Total_I_lwr" : "i", "Total_O_lwr" : "o", "Total_U_lwr" : "u" })
lwr = lwr.T
lwr = lwr.rename(columns = { 0:"Count"})
vows = ['a','e','i','o','u']

lwr['Vowels'] = vows
lwr.reset_index(drop = True, inplace = True)
n = lwr.sum()[0]
lwr = lwr.assign(prop = lambda dataframe: dataframe['Count'].map(lambda Count: (Count / n) * 100 ))
lwr

,Count,Vowels,prop
0,35276,a,21.374342
1,54499,e,33.021892
2,27349,i,16.571235
3,34432,o,20.862948
4,13483,u,8.169584


In [135]:
fig = px.bar(lwr, x = 'Vowels', y = 'Count', color = 'Vowels', color_discrete_sequence = ['#B6E880'] , text =lwr['prop'].apply(lambda x: '{0:1.2f}%'.format(x)))
fig.update_layout(showlegend = False, yaxis_tickformat = ',', title = {'text': 'Distribution of Lower-case Vowels in Sherlock Holmes', 'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top' })
fig.show()

In [129]:
upr = sherlockFinal.iloc[:, 5:10]
upr = upr.rename(columns = {"Total_A_upr" : "A", "Total_E_upr" : "E", "Total_I_Ic" : "I", "Total_O_upr" : "O", "Total_U_upr" : "U"})
upr = upr.T
upr = upr.rename(columns = {0: "Count"})
vows2 = ['A', 'E', 'I', 'O','U']
upr['Vowels'] = vows2
upr.reset_index(drop = True, inplace = True)
n2 = upr.sum()[0]
upr = upr.assign(prop = lambda dataframe: dataframe['Count'].map(lambda Count: (Count / n2) * 100 ))

upr

,Count,Vowels,prop
0,813,A,14.950349
1,332,E,6.105186
2,3843,I,70.669364
3,362,O,6.656859
4,88,U,1.618242


In [132]:
fig = px.bar(upr, x = 'Vowels', color = 'Vowels', color_discrete_sequence = ['#FF6692'], y = 'Count', text =upr['prop'].apply(lambda x: '{0:1.2f}%'.format(x)))
fig.update_layout(showlegend = False, yaxis_tickformat = ',', title = {'text': 'Distribution of Upper-case Vowels in Sherlock Holmes', 'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top' })
fig.show()

## A Tale of Two Cities
'Charles Dickens - Cities.txt'

In [138]:
file_book = open("/spring2021/project1/comparison/Charles Dickens - Cities.txt",'r')

lines = file_book.readlines()[105:]
dataColumns = ["line"]


cityDf = pd.DataFrame(lines, columns = ['Line'])
cityDf2 = spark.createDataFrame(cityDf)

cityDf2.createOrReplaceTempView("city")

new_df2 = spark.sql('SELECT Line, LENGTH(Line)AS LOS, (LENGTH(Line) -  LENGTH(REPLACE(Line,"a",""))) AS a, (LENGTH(Line) -  LENGTH(REPLACE(Line,"e",""))) AS e, (LENGTH(Line) -  LENGTH(REPLACE(Line,"i",""))) AS i, (LENGTH(Line) -  LENGTH(REPLACE(Line,"o",""))) AS o, (LENGTH(Line) -  LENGTH(REPLACE(Line,"u",""))) AS u,(LENGTH(Line) -  LENGTH(REPLACE(Line,"A",""))) AS Ac , (LENGTH(Line) -  LENGTH(REPLACE(Line,"E",""))) AS Ec,(LENGTH(Line) -  LENGTH(REPLACE(Line,"I",""))) AS Ic, (LENGTH(Line) -  LENGTH(REPLACE(Line,"O",""))) AS Oc,(LENGTH(Line) -  LENGTH(REPLACE(Line,"U",""))) AS Uc FROM city')
new_df2.show(10)

+--------------------+---+---+---+---+---+---+---+---+---+---+---+
|                Line|LOS|  a|  e|  i|  o|  u| Ac| Ec| Ic| Oc| Uc|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+
|                   
|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|         CHAPTER I.
| 11|  0|  0|  0|  0|  0|  1|  1|  1|  0|  0|
|         The Period
| 11|  0|  2|  1|  1|  0|  0|  0|  0|  0|  0|
|                   
|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|                   
|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|It was the best o...| 71|  4|  7|  4|  4|  0|  0|  0|  1|  0|  0|
|wisdom, it was th...| 70|  3|  7|  6|  6|  0|  0|  0|  0|  0|  0|
|was the epoch of ...| 69|  4|  6|  5|  4|  1|  0|  0|  0|  0|  0|
|season of Darknes...| 68|  4|  6|  4|  5|  0|  0|  0|  0|  0|  0|
|despair, we had e...| 72|  3| 12|  3|  3|  2|  0|  0|  0|  0|  0|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+
only showing top 10 rows



In [139]:
new_df2.createOrReplaceTempView("vowels")

# aggregate columns
new_df3 = spark.sql('SELECT SUM(a) AS Total_A_lwr, SUM(e) AS Total_E_lwr, SUM(i) AS Total_I_lwr, SUM(o) AS Total_O_lwr, SUM(u) AS Total_U_lwr, SUM(Ac) AS Total_A_upr, SUM(Ec) AS Total_E_upr, SUM(Ic) AS Total_I_Ic, SUM(Oc) AS Total_O_upr, SUM(Uc) AS Total_U_upr  FROM vowels')
new_df3.show()

+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+
|Total_A_lwr|Total_E_lwr|Total_I_lwr|Total_O_lwr|Total_U_lwr|Total_A_upr|Total_E_upr|Total_I_Ic|Total_O_upr|Total_U_upr|
+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+
|      47171|      74230|      38074|      46134|      16635|        927|        441|      2802|        313|         86|
+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+



In [140]:
cityFinal = new_df3.toPandas()
cityFinal

,Total_A_lwr,Total_E_lwr,Total_I_lwr,Total_O_lwr,Total_U_lwr,Total_A_upr,Total_E_upr,Total_I_Ic,Total_O_upr,Total_U_upr
0,47171,74230,38074,46134,16635,927,441,2802,313,86


In [141]:
lwr = cityFinal.iloc[:, 0:5]
lwr = lwr.rename(columns = {"Total_A_lwr": "a", "Total_E_lwr" : "e", "Total_I_lwr" : "i", "Total_O_lwr" : "o", "Total_U_lwr" : "u" })
lwr = lwr.T
lwr = lwr.rename(columns = { 0:"Count"})
vows = ['a','e','i','o','u']

lwr['Vowels'] = vows
lwr.reset_index(drop = True, inplace = True)
n = lwr.sum()[0]
lwr = lwr.assign(prop = lambda dataframe: dataframe['Count'].map(lambda Count: (Count / n) * 100 ))
lwr

,Count,Vowels,prop
0,47171,a,21.224870
1,74230,e,33.400227
2,38074,i,17.131621
3,46134,o,20.758266
4,16635,u,7.485016


In [144]:
fig = px.bar(lwr, x = 'Vowels', y = 'Count', color = 'Vowels', color_discrete_sequence = ['#AB63FA'] , text =lwr['prop'].apply(lambda x: '{0:1.2f}%'.format(x)))
fig.update_layout(showlegend = False, yaxis_tickformat = ',', title = {'text': 'Distribution of Lower-case Vowels in Tale of Two Cities', 'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top' })
fig.show()

In [145]:
upr = cityFinal.iloc[:, 5:10]
upr = upr.rename(columns = {"Total_A_upr" : "A", "Total_E_upr" : "E", "Total_I_Ic" : "I", "Total_O_upr" : "O", "Total_U_upr" : "U"})
upr = upr.T
upr = upr.rename(columns = {0: "Count"})
vows2 = ['A', 'E', 'I', 'O','U']
upr['Vowels'] = vows2
upr.reset_index(drop = True, inplace = True)
n2 = upr.sum()[0]
upr = upr.assign(prop = lambda dataframe: dataframe['Count'].map(lambda Count: (Count / n2) * 100 ))

upr

,Count,Vowels,prop
0,927,A,20.288903
1,441,E,9.652003
2,2802,I,61.326330
3,313,O,6.850514
4,86,U,1.882250


In [148]:
fig = px.bar(upr, x = 'Vowels', color = 'Vowels', color_discrete_sequence = ['#FFA15A'], y = 'Count', text =upr['prop'].apply(lambda x: '{0:1.2f}%'.format(x)))
fig.update_layout(showlegend = False, yaxis_tickformat = ',', title = {'text': 'Distribution of Upper-case Vowels in Tale of Two Cities', 'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top' })
fig.show()

## Pride and Prejudice
'Jane Austen - Pride.txt'

In [155]:
file_book = open("/spring2021/project1/comparison/Jane Austen - Pride.txt",'r')

lines = file_book.readlines()[164:]
dataColumns = ["line"]


cityDf = pd.DataFrame(lines, columns = ['Line'])
cityDf2 = spark.createDataFrame(cityDf)

cityDf2.createOrReplaceTempView("city")

new_df2 = spark.sql('SELECT Line, LENGTH(Line)AS LOS, (LENGTH(Line) -  LENGTH(REPLACE(Line,"a",""))) AS a, (LENGTH(Line) -  LENGTH(REPLACE(Line,"e",""))) AS e, (LENGTH(Line) -  LENGTH(REPLACE(Line,"i",""))) AS i, (LENGTH(Line) -  LENGTH(REPLACE(Line,"o",""))) AS o, (LENGTH(Line) -  LENGTH(REPLACE(Line,"u",""))) AS u,(LENGTH(Line) -  LENGTH(REPLACE(Line,"A",""))) AS Ac , (LENGTH(Line) -  LENGTH(REPLACE(Line,"E",""))) AS Ec,(LENGTH(Line) -  LENGTH(REPLACE(Line,"I",""))) AS Ic, (LENGTH(Line) -  LENGTH(REPLACE(Line,"O",""))) AS Oc,(LENGTH(Line) -  LENGTH(REPLACE(Line,"U",""))) AS Uc FROM city')
new_df2.show(10)

new_df2.createOrReplaceTempView("vowels")

# aggregate columns
new_df3 = spark.sql('SELECT SUM(a) AS Total_A_lwr, SUM(e) AS Total_E_lwr, SUM(i) AS Total_I_lwr, SUM(o) AS Total_O_lwr, SUM(u) AS Total_U_lwr, SUM(Ac) AS Total_A_upr, SUM(Ec) AS Total_E_upr, SUM(Ic) AS Total_I_Ic, SUM(Oc) AS Total_O_upr, SUM(Uc) AS Total_U_upr  FROM vowels')
new_df3.show()

cityFinal = new_df3.toPandas()
cityFinal

lwr = cityFinal.iloc[:, 0:5]
lwr = lwr.rename(columns = {"Total_A_lwr": "a", "Total_E_lwr" : "e", "Total_I_lwr" : "i", "Total_O_lwr" : "o", "Total_U_lwr" : "u" })
lwr = lwr.T
lwr = lwr.rename(columns = { 0:"Count"})
vows = ['a','e','i','o','u']

lwr['Vowels'] = vows
lwr.reset_index(drop = True, inplace = True)
n = lwr.sum()[0]
lwr = lwr.assign(prop = lambda dataframe: dataframe['Count'].map(lambda Count: (Count / n) * 100 ))
lwr

fig = px.bar(lwr, x = 'Vowels', y = 'Count', color = 'Vowels', color_discrete_sequence = ['#FF97FF'] , text =lwr['prop'].apply(lambda x: '{0:1.2f}%'.format(x)))
fig.update_layout(showlegend = False, yaxis_tickformat = ',', title = {'text': 'Distribution of Lower-case Vowels in Pride and Prejudice', 'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top' })
fig.show()

upr = cityFinal.iloc[:, 5:10]
upr = upr.rename(columns = {"Total_A_upr" : "A", "Total_E_upr" : "E", "Total_I_Ic" : "I", "Total_O_upr" : "O", "Total_U_upr" : "U"})
upr = upr.T
upr = upr.rename(columns = {0: "Count"})
vows2 = ['A', 'E', 'I', 'O','U']
upr['Vowels'] = vows2
upr.reset_index(drop = True, inplace = True)
n2 = upr.sum()[0]
upr = upr.assign(prop = lambda dataframe: dataframe['Count'].map(lambda Count: (Count / n2) * 100 ))

upr

fig = px.bar(upr, x = 'Vowels', color = 'Vowels', color_discrete_sequence = ['#EF553B'], y = 'Count', text =upr['prop'].apply(lambda x: '{0:1.2f}%'.format(x)))
fig.update_layout(showlegend = False, yaxis_tickformat = ',', title = {'text': 'Distribution of Upper-case Vowels in Pride and Prejudice', 'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top' })
fig.show()



+--------------------+---+---+---+---+---+---+---+---+---+---+---+
|                Line|LOS|  a|  e|  i|  o|  u| Ac| Ec| Ic| Oc| Uc|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+
|          Chapter 1
| 16|  1|  1|  0|  0|  0|  0|  0|  0|  0|  0|
|                   
|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|      It is a tru...| 67|  6|  4|  4|  1|  2|  0|  0|  1|  0|  0|
|      possession ...| 63|  3|  4|  3|  7|  2|  0|  0|  0|  0|  0|
|                   
|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|      However lit...| 70|  3|  8|  3|  4|  1|  0|  0|  0|  0|  0|
|      on his firs...| 67|  1|  4|  6|  5|  2|  0|  0|  0|  0|  0|
|      fixed in th...| 65|  2|  5|  7|  2|  2|  0|  0|  0|  0|  0|
|      considered ...| 69|  0|  8|  3|  8|  1|  0|  0|  0|  0|  0|
|         daughters.
| 17|  1|  1|  0|  0|  1|  0|  0|  0|  0|  0|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+
only showing top 10 rows

+-----------+-----------+-----------

In [163]:
def makePlots(file_name, colors, title, sp):
    
    file_book = open(file_name,'r')

    lines = file_book.readlines()[sp:]
    dataColumns = ["line"]


    cityDf = pd.DataFrame(lines, columns = ['Line'])
    cityDf2 = spark.createDataFrame(cityDf)

    cityDf2.createOrReplaceTempView("city")

    new_df2 = spark.sql('SELECT Line, LENGTH(Line)AS LOS, (LENGTH(Line) -  LENGTH(REPLACE(Line,"a",""))) AS a, (LENGTH(Line) -  LENGTH(REPLACE(Line,"e",""))) AS e, (LENGTH(Line) -  LENGTH(REPLACE(Line,"i",""))) AS i, (LENGTH(Line) -  LENGTH(REPLACE(Line,"o",""))) AS o, (LENGTH(Line) -  LENGTH(REPLACE(Line,"u",""))) AS u,(LENGTH(Line) -  LENGTH(REPLACE(Line,"A",""))) AS Ac , (LENGTH(Line) -  LENGTH(REPLACE(Line,"E",""))) AS Ec,(LENGTH(Line) -  LENGTH(REPLACE(Line,"I",""))) AS Ic, (LENGTH(Line) -  LENGTH(REPLACE(Line,"O",""))) AS Oc,(LENGTH(Line) -  LENGTH(REPLACE(Line,"U",""))) AS Uc FROM city')
    new_df2.show(10)

    new_df2.createOrReplaceTempView("vowels")

    # aggregate columns
    new_df3 = spark.sql('SELECT SUM(a) AS Total_A_lwr, SUM(e) AS Total_E_lwr, SUM(i) AS Total_I_lwr, SUM(o) AS Total_O_lwr, SUM(u) AS Total_U_lwr, SUM(Ac) AS Total_A_upr, SUM(Ec) AS Total_E_upr, SUM(Ic) AS Total_I_Ic, SUM(Oc) AS Total_O_upr, SUM(Uc) AS Total_U_upr  FROM vowels')
    new_df3.show()

    cityFinal = new_df3.toPandas()
    cityFinal

    lwr = cityFinal.iloc[:, 0:5]
    lwr = lwr.rename(columns = {"Total_A_lwr": "a", "Total_E_lwr" : "e", "Total_I_lwr" : "i", "Total_O_lwr" : "o", "Total_U_lwr" : "u" })
    lwr = lwr.T
    lwr = lwr.rename(columns = { 0:"Count"})
    vows = ['a','e','i','o','u']

    lwr['Vowels'] = vows
    lwr.reset_index(drop = True, inplace = True)
    n = lwr.sum()[0]
    lwr = lwr.assign(prop = lambda dataframe: dataframe['Count'].map(lambda Count: (Count / n) * 100 ))
    lwr
    
    title_final = "Distribution of Lower-case Vowels in " + title

    fig = px.bar(lwr, x = 'Vowels', y = 'Count', color = 'Vowels', color_discrete_sequence = [colors[0]] , text =lwr['prop'].apply(lambda x: '{0:1.2f}%'.format(x)))
    fig.update_layout(showlegend = False, yaxis_tickformat = ',', title = {'text': title_final , 'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top' })
    fig.show()

    upr = cityFinal.iloc[:, 5:10]
    upr = upr.rename(columns = {"Total_A_upr" : "A", "Total_E_upr" : "E", "Total_I_Ic" : "I", "Total_O_upr" : "O", "Total_U_upr" : "U"})
    upr = upr.T
    upr = upr.rename(columns = {0: "Count"})
    vows2 = ['A', 'E', 'I', 'O','U']
    upr['Vowels'] = vows2
    upr.reset_index(drop = True, inplace = True)
    n2 = upr.sum()[0]
    upr = upr.assign(prop = lambda dataframe: dataframe['Count'].map(lambda Count: (Count / n2) * 100 ))

    upr
    title_final = "Distribution of Upper-case Vowels in " + title

    fig = px.bar(upr, x = 'Vowels', color = 'Vowels', color_discrete_sequence = [colors[1]], y = 'Count', text =upr['prop'].apply(lambda x: '{0:1.2f}%'.format(x)))
    fig.update_layout(showlegend = False, yaxis_tickformat = ',', title = {'text': title_final, 'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top' })
    fig.show()

    

In [170]:
file_name = "/spring2021/project1/comparison/Scott Fitzgerald - Gatsby.txt"
colors = ['rgb(141,211,199)', 'rgb(249, 123, 114)']
title = "The Great Gatsby"
startingPoint = 45
makePlots(file_name, colors,title , startingPoint)

+--------------------+---+---+---+---+---+---+---+---+---+---+---+
|                Line|LOS|  a|  e|  i|  o|  u| Ac| Ec| Ic| Oc| Uc|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+
|                 ...| 41|  2|  1|  1|  0|  0|  0|  0|  0|  1|  0|
|                 ...| 37|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|
|                 ...| 39|  1|  1|  0|  0|  0|  0|  0|  0|  0|  0|
|                   
|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|  Then wear the g...| 49|  3|  5|  2|  2|  0|  0|  0|  0|  0|  0|
|  If you can boun...| 46|  1|  3|  1|  6|  3|  0|  0|  1|  0|  0|
|  Till she cry “L...| 57|  1|  4|  3|  4|  1|  0|  0|  0|  0|  0|
|  I must have you!”
| 20|  1|  1|  0|  1|  2|  0|  0|  1|  0|  0|
|                   
|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|  Thomas Parke d’...| 28|  2|  2|  2|  1|  0|  0|  0|  1|  0|  0|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+
only showing top 10 rows

+-----------+-----------+-----------

## Hacker News

In [ ]:
#blogtext.csv
file_name = "/spring2021/project1/blogtext.csv"
colors = ['#FF9DA6', '#9D755D']
title = "The Blog Text"
startingPoint = 2
makePlots(file_name, colors,title , startingPoint)

+--------------------+-----+----+----+----+----+---+---+---+---+---+---+
|                Line|  LOS|   a|   e|   i|   o|  u| Ac| Ec| Ic| Oc| Uc|
+--------------------+-----+----+----+----+----+---+---+---+---+---+---+
|2059027,male,15,S...|  226|  12|  16|   9|   1|  6|  1|  0|  0|  0|  0|
|2059027,male,15,S...|25512|1508|2300|1356|1664|827| 66| 31| 69| 12| 40|
|2059027,male,15,S...|   88|   2|   5|   2|   1|  1|  0|  0|  0|  0|  0|
|3581210,male,33,I...|  463|  19|  28|  21|  40| 17|  1|  1|  3|  0|  1|
|3581210,male,33,I...| 3756| 205| 370| 147| 234| 84|  6|  2| 15|  2|  8|
|3581210,male,33,I...| 1166|  66|  93|  59|  62| 29|  2|  0|  7|  1|  0|
|3581210,male,33,I...| 2342| 159| 244| 109| 126| 39|  5|  5|  6|  1|  1|
|3581210,male,33,I...| 2350| 143| 210| 100| 159| 54|  4|  2| 14|  3| 10|
|3581210,male,33,I...| 1018|  57|  87|  53|  58| 19|  5|  2|  6|  0|  0|
|3581210,male,33,I...| 1828| 104| 153| 100| 101| 56|  7|  3| 11|  3|  1|
+--------------------+-----+----+----+----+----+---